# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

🙏🏻_Антон, спасибо за ревью и комментарии! Действительно, некоторые моменты упустил. Постараюсь в предь быть более аккуратным._

## Changelog 17.09.2019

> 1. Удалять колонки с total_income NaN вот так вот сразу – достаточно лихое действие 🙂 Ты уверен, что хочешь удалять 10% датасета до сразу? Кажется, это требуется только для решения одной из следующих задач.
> 2. Я бы еще тут поисследовал, насколько случайны пропуски. Например, посмотрел бы как распределены пропуски в разных колонках.

Видимо мой мозг разработчика работает по принципу "меньше кода – меньше багов", но этот принцип не для данных 😀Как я писал ниже, пропуски не случайны и признаки `days_employed` и `total_income` взаимосвязаны. У меня было два варианта восстановления пропусков:

1. Заполнить пропуски 0
2. Заполнить пропуски медианой по каждому из столбцов

✔ Т.к. я пока не знаю когда лучше применять второй вариант и насколько он окажется эффективным, то остановился на первом.

----

```python
# Определим основные категории целей кредита
purpose_category = [
    'свадьба',
    'недвижимость',
    'жилье',
    'автомобиль',
    'образование'
]
```
> Как получен этот список? Представь, что у тебя значительно больше уникальны причин – ты же не отсмотришь их все глазами. Здесь все-таки стоит придумать какой-то автоматический способ детектировать топовые леммы

✔️ Верно, если значений будет много, то станет проблематично быстро и без ошибок выделять категории. Альтернативным решением может быть поиск самых частовстречающихся слов в леммах и использование их в качестве категорий. Добавил этот момент.

----

> 2 – все-таки малодетная или многодетная? Кажется, код неоднозначный, дальше в .head() даже видно пример бага

✔ fixed: ключ для `малодетная` должен быть `2`. Кстати, пишут ли юнит-тесты для маленьких функций в рамках предобработки данных?

---

> Откуда здесь взяты числа для разделения? Тут лучше плясать от данных. Вдруг это какой-нибудь крайне специфический банк и там 99.9% с доходом <60к?

✔️Проверил разброс по доходу и выделил медиану. На основе этого сформировал категории.

---

> Шаг 3. Ответьте на вопросы

✔️Скорректировал выводы используя сводные таблицы.

## Шаг 1. Откроем файл с данными и изучим общую информацию

In [1]:
import pandas as pd
import numpy as np

customer_solvency = pd.read_csv('/datasets/data.csv')

### Выведем общую информацию о датафрейме и посмотрим на типы данных

In [2]:
customer_solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Проверим наличие дубликатов

In [3]:
customer_solvency.duplicated().sum()

54

### ... и пропусков

In [4]:
customer_solvency.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

### Посмотрим на первые и последние 5 записей

In [5]:
customer_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [6]:
customer_solvency.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


### Пробежимся по всем колонкам и найдем странности

In [7]:
# Количество детей
customer_solvency.sort_values('children')['children'].value_counts().sort_index()

-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

In [8]:
# Общий трудовой стаж в днях. Отрицательные значения!
customer_solvency[customer_solvency['days_employed'] < 0]['days_employed'].count()

15906

In [9]:
# Положительные значения слишком большие для количества дней, даже для людей приклонного возраста
customer_solvency[customer_solvency['days_employed'] > 0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью


In [10]:
# Возраст
customer_solvency.sort_values('dob_years')['dob_years'].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [11]:
# Образование
customer_solvency.sort_values('education')['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [12]:
# Семейное положение
customer_solvency['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [13]:
# Пол
customer_solvency['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [14]:
# Тип занятости
customer_solvency['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [15]:
# Проверим есть ли люди с нулевым уровнем дохода
customer_solvency[customer_solvency['total_income'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [16]:
# А что насчет пропусков?
customer_solvency[customer_solvency['total_income'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [17]:
# Цель получения кредита
customer_solvency['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

* Наш датасет состоит из _21 525_ записей;
* Из них 54 дубликата – не много, однако это сырые данные и количество дубликатов может измениться после базовой предобработки;
* Одинаковое количество пропусков в столбцах `days_employed` и `total_income` – _2 174_. Связь между этими столбцами понятна: невозможно рассчитать общий доход не имея информации о стаже. Не все компании и люди работают по ТК РФ 😔Так как столбец `total_income` нам нужен для ответа на вопрос _"Есть ли зависимость между уровнем дохода и возвратом кредита в срок?"_ , то от этих записей пользы нет. Тем более такая категория клиентов – **high risk** для банка.
* 🆕Попробуем восстановить пропуски заменив NaN на `0`.
* Столбец `children`:
    * Для 47 записей количество детей равно `-1` – т.к. у нас нет возможности связаться с коллегами, которые сделали нам выгрузку датасета, и причину появления таких значений установить сложно (например, клиенты банка просто не захотели делиться такой информацией), будем считать их нерепрезентативными и удалим эти записи. Их всего 47, и они не должны коренным образом повлиять на анализ;
    * 76 человек имеет по 20 детей 😲Выглядит как ошибка в данных, у нас же не 🐰Снова, мы могли бы предположить, что корретное значение – это `2`, а `0` появился при неправильном заполнении данных. Но это как гадать на кофейной гуще – можно, но не факт, что сбудется. Удаляем;
* Столбец `days_employed` имеет тип `float64` и содержит отрицательные значения. Всего таких записей _15 906_ , т.е. **~73%** от общего числа!
    * Будем работать с днями, поэтому приведем тип столбца к `int64`;
    * Положительные значения из датасета выглядят странными. Например, 343937.404131 / 365 дней = 942 лет стажа❗️Хорошо, что в решении бизнес задачи этот столбец нам не пригодится. В противном случае пришлось бы идти к коллегам рассказывать об ошибке в выгрузке данных и выяснять причину появления ошибок 💡
    * А вот отрицательные значения (потеряно дней жизни? 😃) сделаем абсолютными, т.к. эти данные выглядят правдоподобнее. Даже свою функцию писать не придется – применим стандартную `abs`. Сделаем это для тренировки и улучшения читабельности данных 🤓
* Столбец `dob_years` – для 101 человека возраст неизвестен. Для ответа на вопросы бизнеса этот стоблец не используется. Возьмем на заметку и оставим как есть ❗️
* В столбце `education` встречаются категории в разном регистре – нужно будет привести к единому виду, например, все в lowercase. И еще раз проверить на дубликаты. Предварительно убедимся в том, что `education_id` соотносится с нужным типом образования;
* В столбце `family_status` только одна категория `Не женат / не замужем` выбивается из общей семантики именования – начинается с заглавной буквы;
* У одного клиента неизвестен пол – для анализа этот столбец мы не используем, поэтому пока оставим эту запись;
* Столбец `purpose` содержит разные формулировки одних и тех же целей – применим лемматизацию.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [18]:
# Заполним значения NaN в столбце 'days_employed' нулями
customer_solvency['days_employed'] = customer_solvency['days_employed'].fillna(0)

In [19]:
# Заполним значения NaN в столбце 'total_income' нулями
customer_solvency['total_income'] = customer_solvency['total_income'].fillna(0.0)

In [20]:
# Избавляемся от записей, в которых есть ошибочные данные
customer_solvency = customer_solvency[(customer_solvency['children'] >= 0) & (customer_solvency['children'] <= 5)]
customer_solvency.sort_values('children')['children'].value_counts().sort_index()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

### Вывод

In [21]:
customer_solvency.shape[0]

21402

Мы избавились от пропусков, оставив только необходимые для будущего анализа данные.

### Замена типа данных

In [22]:
customer_solvency['days_employed'] = customer_solvency['days_employed'].astype('int').apply(abs)

In [23]:
customer_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Преобразовали количество дней стажа в целочисленный тип при помощи метода `astype`, т.к. `to_numeric` служит для конвертации строк и приводит их к числам с плавающей точкой. Дополнительно столбец избавился от отрицательных значений.

### Обработка дубликатов

In [24]:
# Приведем значения в столбце Уровень образования клиента к единому регистру
customer_solvency['education'] = customer_solvency['education'].str.lower()

In [25]:
customer_solvency.sort_values('education')['education'].value_counts()

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

In [26]:
customer_solvency.duplicated().sum()

71

### Вывод

После нормализации значений колонки `education` необходимость в удалении дубликатов при помощи метода `drop_duplicates()` отпала. Их количество равно 0.

### Лемматизация

In [ ]:
from collections import Counter

# Воспользуемся библиотекой Mystem от Яндекса
from pymystem3 import Mystem
m = Mystem()

# Попробуем найти самые частовстречаемые слова в леммах и на основе результата определить категории целей кредита
purposes = []

for p in customer_solvency['purpose']:
    lemma = m.lemmatize(p)
    purposes += filter(lambda l: l not in [' ', '\n', 'с', 'со', 'на'], lemma)

print(Counter(purposes))

In [ ]:
# В приведенном выше множестве мы вимим общие слова типа "покупка", "операция", "строительство" и пр.
# Если их откинуть, то наиболее частовстречаемыми категориями будут:
purpose_category = [
    'свадьба',
    'недвижимость',
    'жилье',
    'автомобиль',
    'образование'
]

In [ ]:
# Опишем функцию, которую будем вызывать при добавлении нового стобца
def categorize_purpose(text):
    """
    Функция возвращает категорию цели креди на основе лемматизированного текста.
    
    P.S. Поиск вхождения категории в лемму можно реализовать более оптимально через пересечение множеств.
    Тем не менее имплементация с вложенными циклами или list comprehensions более явная.
    """
    
    lemmas = m.lemmatize(text)
    for lemma in lemmas:
        for category in purpose_category:
            if category in lemma:
                return category
        

# "Протестируем" функцию
categorize_purpose('Ах эта свадьба, свадьба, свадьба пела и плясала...')

In [ ]:
# Наконец, добавим новый столбец с категориями целей кредита
customer_solvency['purpose_category'] = customer_solvency['purpose'].apply(categorize_purpose)

In [ ]:
customer_solvency.head()

### Вывод

Из большого числа вариаций получили всего пять категорий целей кредита – читать и анализировать такие данные намного проще.

### Категоризация данных

Для ответа на нижеперечисленные вопросы нам необходимо сегментировать пользователей. Четыре вопроса – четыре раза выполним категоризацию.

> Есть ли зависимость между наличием детей и возвратом кредита в срок?

Распределим клиентов по наличию детей:

* 0 – бездетная
* 1 – однодетная
* 2 – малодетная
* 3+ – многодетная

In [ ]:
def categorize_children(count):
    categories = {
        0: 'бездетная',
        1: 'однодетная',
        2: 'малодетная'
    }

    return categories.get(count, 'многодетная')

In [ ]:
customer_solvency['children_category'] = customer_solvency['children'].apply(categorize_children)

In [ ]:
customer_solvency.head()

> Есть ли зависимость между семейным положением и возвратом кредита в срок?

По семейному положению – вопрос интересный. Всего у нас пять категорий (см. `Out[12]`), но давайте сузим запрос и возьмем за основу `family_status_id` до следующих категории:

* "в браке" – женат / замужем, гражданский брак
* "не в браке" – не женат / не замужем, в разводе, вдовец / вдова

In [ ]:
def categorize_family_status(value):
    if value == 'женат / замужем':
        return 'в браке'
    
    return 'не в браке'

In [ ]:
customer_solvency['family_status_category'] = customer_solvency['family_status'].apply(categorize_family_status)

In [ ]:
customer_solvency.head()

> Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Уровень дохода клиента лучше рассматривать в рамках определенного города или региона, так как уровень жизни и зарплаты могут сильно отличаться, например, в Москве и Брянске. Но у нас подобных данных в датасете нет.

In [ ]:
customer_solvency.hist(column='total_income', bins=100)

Т.к. мы восстановили пропуски, то минимальное значение признака `total_income` равно `0`. Найдем таже медиану и максимальное значение.

In [ ]:
timean = customer_solvency['total_income'].mean()

In [ ]:
timean

In [ ]:
customer_solvency['total_income'].max()

In [ ]:
def categorize_total_income(value):
    if value < 50_000:
        return 'низкий'
    elif value <= timean:
        return 'средний'
    elif value < 350_000:
        return 'выше среднего'
    else:
        return 'высокий'

In [ ]:
customer_solvency['total_income_category'] = customer_solvency['total_income'].apply(categorize_total_income)

In [ ]:
customer_solvency.tail()

> Как разные цели кредита влияют на его возврат в срок?

Цели кредита мы уже категоризовали выше на этапе лемматизации:

* свадьба
* недвижимость
* жилье
* автомобиль
* образование

### Вывод

Мы категоризовали данные. Теперь, группируя данные по нашим категориям, мы сможем ответить на вопросы ниже.

### Шаг 3. Ответьте на вопросы

#### 1. Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
customer_solvency.loc[:, ['children_category', 'debt']].groupby('children_category')['debt'].value_counts()

In [ ]:
pd.pivot_table(customer_solvency, index='children_category', values='debt')

#### Вывод

* Прямой зависимости между наличием детей и возвратом кредита в срок нет: до `9.4%` клиентов имели задолжности по кредитам, независимо от того были у них дети или нет.
* Бездетные семьи чаще выплачивают кредит (ожидаемо): `~7.5%`.
* Что интересно, многодетные семьи реже остаются в должниках `~8.2%`, чем семьи с одним ребенком `~9.2%` или двумя `~9.4%`.

#### 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
pd.pivot_table(customer_solvency, index='family_status_category', values='debt')

#### Вывод

Клиенты, состоящие в браке, на `1.3%` реже остаются в должниках.

#### 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
pd.pivot_table(customer_solvency, index='total_income_category', values='debt')

#### Вывод

* Как и ожидалось, меньше всего в должниках остаются люди с высоким уровнем дохода: `~6.4%`.
* Заметной разницы между клиентами со средним и выше среднего уровнями дохода нет: `~8.4%` против `~8%` должников.
* Что интересно, люди с низким уровнем достатка остаются в должниках реже, чем более состоятельные `~7.6%`.

_Таким образом, мы не можем сказать, что чем выше доход клиента, тем ниже уровень риска при выдаче кредита. Но можем выделить 2 категории: люди с низким и высоким дохода 🤔_

⚠️ Напомню, что в наших данных были пропуски, и мы заполнили их `0`. Поэтому полностью полагаться на данный вывод мы не можем.

#### 4. Как разные цели кредита влияют на его возврат в срок?

In [ ]:
pd.pivot_table(customer_solvency, index='purpose_category', values='debt')

#### Вывод

Посчитав % задолжностей по категориям, можно сделать вывод, что автомобиль `9.3%` и образование `9.2%` не на первом месте в списке приоритетов на выплаты у клиентов, в отличие от жилья `~6.9%`, недвижимости `7.5%` и свадьбы `7.8%`.

### Шаг 4. Общий вывод

Исходя из предыдущих выводов, мы можем сделать следующую рекомедацию бизнесу по выдаче кредита: безопаснее всего выдавать кредиты на покупку жилья, недвижимости или свадьбу – по ним меньшее количество задолжностей.

Для более детального ответа необходимо рассмотреть следующие факты в совокупности:

* Клиенты без детей 😦
* Семейные ☝️
* С высоким уровнем дохода 💵
* Цель кредита – жилье или недвижимость 🎯

Возможно это идеальный портрет клиента.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.